In [1]:
from twython import Twython
from twython import TwythonStreamer

In [2]:
import re
import pandas as pd
from collections import OrderedDict

In [3]:
api = Twython('BKo9wF0sGMYzCTqKmKwnL16BY','DVTqCsvcDY0q3f4dDQ8LlrUr4P13OkpmCYJfQfNw6lpgPTiBCt',
                '844835244565970944-qMaMnAb8smEEROnbXvb2Rw7Rqnazk9g','LcK3mgBLDBW5xBT4O7BZaa1ZO6WuFt4YWtPnn2PqC8WY0')

In [6]:
result = api.search(q='trump',count='3',until = '2018-03-21')
result['search_metadata']

{'completed_in': 0.024,
 'count': 3,
 'max_id': 976247011530899456,
 'max_id_str': '976247011530899456',
 'next_results': '?max_id=976247011270971391&q=trump%20until%3A2018-03-21&count=3&include_entities=1',
 'query': 'trump+until%3A2018-03-21',
 'refresh_url': '?since_id=976247011530899456&q=trump%20until%3A2018-03-21&include_entities=1',
 'since_id': 0,
 'since_id_str': '0'}

In [7]:
n = 1
for tweet in result['statuses']:
    print('No: %s ; ID: %s' % (n,tweet['id']))
    print("Language: %s" % (tweet['lang']))
    print("Tweet Date: %s" % (tweet['created_at']))
    print(tweet['text'],"\n")
    n += 1

No: 1 ; ID: 976247011530899456
Language: en
Tweet Date: Tue Mar 20 23:59:59 +0000 2018
RT @RVAwonk: Trump once called Fox News legal analyst Andrew Napolitano a "great legal mind."

Today, Napolitano said the firing of former… 

No: 2 ; ID: 976247011296055302
Language: en
Tweet Date: Tue Mar 20 23:59:59 +0000 2018
RT @AdamSchiffCA: That Donald Trump would be a deeply flawed President was all too predictable. But that he could remake an entire party in… 

No: 3 ; ID: 976247011270971392
Language: en
Tweet Date: Tue Mar 20 23:59:59 +0000 2018
@JeffFlake We are begging the GOP to stop being complicit spineless power hungry traitors. Change now or you will w… https://t.co/Iy8NYSQnVi 



In [8]:
result['search_metadata']['next_results'].split('max_id=')[1].split('&')[0]

'976247011270971391'

In [9]:
tweet_list = []
try_numbers = 2
n = 0

for i in range(0,try_numbers):
    if i == 0:
        result = api.search(q='trump',count='40',until ='2018-03-21')
    else:
        try:
            result = api.search(q='trump',count='100',include_entities='true',max_id=next_max_id)
        except:
            print('Twitter API Rate limit exceeded.')
            time.sleep(1000)
            continue

    for tweet in result['statuses']:
        tweet_d = OrderedDict()
        tweet_d['time'] = pd.to_datetime(tweet['created_at'])
        tweet_d['content'] = tweet['text']
        tweet_list.append(tweet_d)
    n += 1
    print('page',n,'available')
    try:
        # Parse the data returned to get max_id to be passed in consequent call.
        next_results_url_params = result['search_metadata']['next_results']
        next_max_id = next_results_url_params.split('max_id=')[1].split('&')[0]
    except:
        print('No more next pages.')
        break

page 1 available
page 2 available


In [10]:
tweet_list[:3]

[OrderedDict([('time', Timestamp('2018-03-20 23:59:59')),
              ('content',
               'RT @RVAwonk: Trump once called Fox News legal analyst Andrew Napolitano a "great legal mind."\n\nToday, Napolitano said the firing of former…')]),
 OrderedDict([('time', Timestamp('2018-03-20 23:59:59')),
              ('content',
               'RT @AdamSchiffCA: That Donald Trump would be a deeply flawed President was all too predictable. But that he could remake an entire party in…')]),
 OrderedDict([('time', Timestamp('2018-03-20 23:59:59')),
              ('content',
               '@JeffFlake We are begging the GOP to stop being complicit spineless power hungry traitors. Change now or you will w… https://t.co/Iy8NYSQnVi')])]

In [11]:
data3 = pd.DataFrame(tweet_list)

In [12]:
tag_list = []
tag_dict = {}
for each in data3['content']:
    each = each.lower()
    tags = re.findall(r'#[\w]+',each)
    for tag in tags:
        if tag not in tag_dict:
            tag_dict['%s' %tag] = 1
        else:
            tag_dict['%s' %tag] += 1

tag_list.append(tag_dict)

In [13]:
tag_frame = pd.DataFrame(tag_list)
tag_frame = pd.DataFrame.transpose(tag_frame).reset_index()
tag_frame.columns = ['tag','freq']
tag_frame.sort_values(by = 'freq',ascending = False)

,tag,freq
20,#trump,4
0,#bloomberg,1
1,#cambridgeanalytica,1
19,#stormydanielssavesame,1
18,#slowwalker,1
17,#qanon,1
16,#pizz,1
15,#pedogate,1
14,#opioidepidemic,1
13,#nationalsecurity,1
